In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn import preprocessing

import csv

CSV_FILE = 'scrap_dataset.csv'

In [ ]:
"""
with open(CSV_FILE,'r') as csv_file:
    csv_reader = csv.Reader(csv_file)
    line_count = 0
    for row in csv_reader:
        line_count+=1
        if line_count == 1:
            continue
        if row[]
"""

In [121]:
dataset = pd.read_csv(CSV_FILE)



In [117]:
features = ['text', 'font_size', 'font_color_red', 'font_color_green' , 'font_color_blue' , 'font_weight']

In [122]:
dset = dataset.drop(['font_color_alpha'], axis=1)
# dset = dataset.drop('text', axis=1)
le = preprocessing.LabelEncoder()
dset['text'] = le.fit_transform(dset['text'])

dset.drop_duplicates(inplace=True, keep=False)

In [123]:
dset2 = dset
cpy = dset2

dset2 = dset2.drop('output', axis=1)
X_train, X_test, y_train, y_test = train_test_split(dset2, cpy['output'], test_size=0.35, random_state=123)

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# SVM without sampling
y_pred = svm.predict(X_test)

# Evaluate the performance of the classifier
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.55      0.31      0.39       157
           1       0.81      0.92      0.86       498

    accuracy                           0.77       655
   macro avg       0.68      0.61      0.63       655
weighted avg       0.75      0.77      0.75       655



In [124]:

#Down Sampling majority
df_majority = dset[dset.output == 1]
df_minority = dset[dset.output == 0]

#df_majority_downsampled = resample(df_majority, replace=False,  n_samples=len(df_minority) , random_state=123) # reproducible results
df_minority_downsampled = resample(df_minority, replace=True,  n_samples=int(len(df_majority)*0.6) , random_state=123) # reproducible results

#dset = pd.concat([df_majority_downsampled, df_minority])
dset = pd.concat([df_minority_downsampled, df_majority])

cpy = dset
dset = dset.drop('output', axis=1)

X_train, X_test, y_train, y_test = train_test_split(dset, cpy['output'], test_size=0.35, random_state=123)



In [125]:
# With Upsampling Minority

svm = SVC(kernel='poly')
svm.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = svm.predict(X_test)

# Evaluate the performance of the classifier
print(classification_report(y_test, y_pred, zero_division=0))


              precision    recall  f1-score   support

           0       0.65      0.45      0.53       303
           1       0.72      0.85      0.78       502

    accuracy                           0.70       805
   macro avg       0.68      0.65      0.65       805
weighted avg       0.69      0.70      0.68       805



In [126]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rfc.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7031055900621118


In [131]:
dtc = DecisionTreeClassifier(random_state=42)
k = 5 # Set the number of folds
scores = cross_val_score(dtc, dset, cpy["output"], cv=k)

# Print the results
print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})")

# Without Cross Validation

dtc.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = dtc.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.73 (+/- 0.32)
Accuracy: 0.7279503105590062


In [132]:
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize an array to store the accuracy scores for each fold
scores = []
X = dset
y = cpy['output']
# Iterate over the k folds
for train_index, test_index in kf.split(X):
    
    # Split the data into train and test sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize the decision tree classifier
    dtc = DecisionTreeClassifier(random_state=42)
    
    # Fit the classifier on the training data
    dtc.fit(X_train, y_train)
    
    # Predict the target variable on the test data
    y_pred = dtc.predict(X_test)
    
    # Calculate the accuracy score for the fold and append it to the scores array
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Calculate the mean and standard deviation of the accuracy scores
mean_score = sum(scores) / k
std_dev = (sum((score - mean_score) ** 2 for score in scores) / k) ** 0.5

# Print the results
print(f"Accuracy: {mean_score:.2f} (+/- {std_dev:.2f})")

Accuracy: 0.71 (+/- 0.00)
